The file is a guided tour of portfolio_allocator's functional components.  Together, we will see how portfolio_allocator preporcesses data and employs two methods to calculate allotments.  The code herein also allows users to randomly generate different input configurations to see how portfolio_allocator performs across different scenarios.  For a detailed explaination of the purposes and uses of portfolio_allocator, please see the file included in the project repository, purpose_and_uses.md.

In [1]:
#import relevant libraries
import pandas as pd
import numpy as np
from random import randint
import sympy as sp

# Inputs
Following the import of relevant libraries, 4 basic inputs are defined:
- `cash`
- `number_of_investments`
- `target_pcts`
- `current_values`

Although not necessary when using portfolio_allocator, `number_of_investments` is included below to enable the generation of random values.

The utility of the portfolio allocation methods will vary depending on these inputs.  Sometimes, portfolio allocation is so straightforward, advanced methods are not needed.  Other times, portfolio allocation can require advanced methods.  To appreciate this, look at the existing output before running any code.  You will see a scenario in which Method 1 is suboptimal and Method 2 is required to achieve an optimal allocation.  To explore other scenarios, the existing code will generate random values for `cash` and `current_values`.  

To explore even further, the current default values for `number_of_investments` and `target_pcts` can be replaced with the commented out code, however; note that such variety will often generate uninteresting scenarios.  The methods presented here are most useful when there are significant discrepancies between current and target values across many but not all investments, and there is a large amount of `cash` to allocate but not enough meet all target values.

In [6]:
# Define the basic features of the portfolio and investment

# Cash is the lump sum to be alloted to investments
cash = np.random.randint(1000,10000); print('cash = ',cash)

# The default inputs used here capture the set of conditions that make the useuflness of portfolio_allocator most obvious
# However, the script will work with random values (generated by the commented out code) for the sake of experimentation
number_of_investments = 10 #np.random.randint(3,20)

# target_pcts are the set of target allocations specified as proportions that sum to 1 (minus rounding error)
def create_target_pcts(n_i):
    n = 20 - n_i
    arr = [1] * n_i
    for i in range(n):
        arr[randint(0, n) % n_i] += 1
    return [(5*x)/100 for x in arr]
target_pcts = [.1,.1,.1,.1,.1,.1,.1,.1,.1,.1]#create_targets(number_of_investments)
assert round(sum(target_pcts)) == 1, 'sum of target % allocations do not equal zero'

# current_values are the current values of the investments in the portfolio
current_values = np.random.randint(10,1000, size=number_of_investments)

# Create DataFrame
df = pd.DataFrame({'target%':target_pcts,'current_value':current_values})
df

cash =  2060


,target%,current_value
0,0.1,796
1,0.1,29
2,0.1,29
3,0.1,415
4,0.1,961
5,0.1,466
6,0.1,217
7,0.1,12
8,0.1,848
9,0.1,910


# Data Preparation

In [7]:
# Create calculated columns which will be used by the allocation methods below.
    # note that target values reflect cash as shown in calculation
df['current%'] = df['current_value'] / df['current_value'].sum()
df['target_value'] = df['target%'] * (df['current_value'].sum()+cash)
df['deficit'] = df['target_value'] - df['current_value']
df['error'] = (df['current_value'] / df['current_value'].sum()) - df['target%']

# create 'rank column' to reflect rankings of deficits (i.e., discrepancies between the target and current values)
    # rankings are made in descending order such that the largest deficit gets a rank of 0
df['rank'] = df['deficit'].rank(method='first',ascending=False).astype(int)-1

# having the dataframe sorted by rank makes calculations subsequent easier to interpret
df.sort_values(by='rank',inplace=True)
df

,target%,current_value,current%,target_value,deficit,error,rank
7,0.1,12,0.002562,674.3,662.3,-0.097438,0
1,0.1,29,0.006193,674.3,645.3,-0.093807,1
2,0.1,29,0.006193,674.3,645.3,-0.093807,2
6,0.1,217,0.046338,674.3,457.3,-0.053662,3
3,0.1,415,0.088618,674.3,259.3,-0.011382,4
5,0.1,466,0.099509,674.3,208.3,-0.000491,5
0,0.1,796,0.169977,674.3,-121.7,0.069977,6
8,0.1,848,0.181081,674.3,-173.7,0.081081,7
9,0.1,910,0.194320,674.3,-235.7,0.094320,8
4,0.1,961,0.205210,674.3,-286.7,0.105210,9


# Method 1: allocate-by-rank

This method allots the cash to completely eliminate the deficits in rank order untill the cash is gone.  For example, if the cash is \\$3 and the two top ranking deficits are each \\$2, 
this method will result in an allocation of \\$2 to the top ranking deficit and \\$1 to the subsequent deficit.

In [8]:
# define function to determine allotments based on the index of the DataFrame and the amount alloted so far
    # The function allots an amount equal to the deficit or whatever dollars remain 
    # after accounting for previous allocations
    
def determine_amount(i,allocated_so_far):
    if (df.loc[i,'deficit'] <= cash - allocated_so_far) & (df.loc[i,'deficit']>=0): 
        return df.loc[i,'deficit']
    elif (df.loc[i,'deficit'] >= cash - allocated_so_far) & (df.loc[i,'deficit']>=0): 
         return cash - allocated_so_far
    elif df.loc[i,'deficit'] < 0:
        return 0

# calculate the allotment for each investment
total_allocation = 0
money = cash
allocated_cumsum = 0
for rank in df['rank']:
    b = df['rank']==rank
    index = df.loc[b].index[0]
    allocation = determine_amount(index,allocated_cumsum)
    df.loc[b,'allocate_by_rank'] = allocation
    allocated_cumsum += allocation
    money = cash - allocated_cumsum

# calculate the error of the new values of investments in the hypothetical portfolio in which these allotments were made
df['error2'] = ((df['current_value'] + df['allocate_by_rank']) / (df['current_value'].sum()+cash)) - df['target%']

# make sure the sum of the allocations equals the cash
print('cash = ',cash)
assert round(df['allocate_by_rank'].sum()) == cash, "sum of Method 1 allocations does not equal cash"

df

cash =  2060


,target%,current_value,current%,target_value,deficit,error,rank,allocate_by_rank,error2
7,0.1,12,0.002562,674.3,662.3,-0.097438,0,662.3,0.000000
1,0.1,29,0.006193,674.3,645.3,-0.093807,1,645.3,0.000000
2,0.1,29,0.006193,674.3,645.3,-0.093807,2,645.3,0.000000
6,0.1,217,0.046338,674.3,457.3,-0.053662,3,107.1,-0.051935
3,0.1,415,0.088618,674.3,259.3,-0.011382,4,0.0,-0.038455
5,0.1,466,0.099509,674.3,208.3,-0.000491,5,0.0,-0.030891
0,0.1,796,0.169977,674.3,-121.7,0.069977,6,0.0,0.018048
8,0.1,848,0.181081,674.3,-173.7,0.081081,7,0.0,0.025760
9,0.1,910,0.194320,674.3,-235.7,0.094320,8,0.0,0.034955
4,0.1,961,0.205210,674.3,-286.7,0.105210,9,0.0,0.042518


# Method 2: allocate-for-minimal-errors

Method 1 can lead to suboptimal results.  When alloting cash to elminate deficits, 
errors are reduced to zero for investments with the top ranking deficits, 
but may remain large for investments with lower ranking deficits.  The optimal result would be a set of allotments that minimize all errors as much as possible.  
Method 2 achieves this by calculating the minimum possible error for all investments with a deficit greater than 0,
and then calculating the allotments required to produce those errors.  

However, Method 2 can return impractical allotments when not needed.  That is, when all error values for investments with deficits greater than zero are minimized by Method 1, Method 2 may return a set of allotments that includes negative numbers.  Therefore, is it important to examine the error values produced by Method 1 before proceeding to Method 2.

In [9]:
# check to see if method 2 will be useful or not
b1 = df['deficit'] >= 0
b2 = df['allocate_by_rank'] > 0
if df.loc[b1&b2,'error2'].abs().min() < df.loc[b1&~b2,'error2'].abs().max():
    print('Method 1 is suboptimal.')
else:
    print('Method 2 is unnecessary and may return an allocation set that includes negative numbers.')

Method 1 is suboptimal


In [12]:
# For illustrate purposes, create system of equations required to calculate the minimum equal error 
# achievable for investments with a deficit greater than 0
    
# The first set of equations are derived from the same equation used to calculate the "error" columns above
    # the symbol e is reserved for error
    # all other single, lowercase, letter symbols represent allocations
cv = df['current_value'].to_list()
t_pct = df['target%'].to_list()
from sympy.abc import *
equations = {}
allotment_symbols = []
b = df['deficit'] >= 0
deficit_cnt = df.loc[b,'rank'].max()
for I,S in enumerate('abcdfghijklmnopqrstuvwxyz'[:deficit_cnt+1]):
    allotment_symbols.append(symbols(S))
    equations.update({'eq'+str(I):(((cv[I] + symbols(S)) / (sum(cv)+cash)) - t_pct[I]) - e})

# The final equation reflects the constraint that the sum of all allotments should equal the cash available to invest
equations.update({'eq'+str(deficit_cnt+1):sum(allotment_symbols) - cash})
print('\nSystem of equations: '); display(equations)

# solve the system of equations
symbol_values = sp.solve(equations.values(), allotment_symbols+[e])

# make sure that the sum of the allocations is equal to the cash available to allocate
assert sum([symbol_values[S] for S in symbol_values if S != e]) == cash, 'sum of allocations != cash'


System of equations: 


{'eq0': a/6743 - e - 0.0982203766869346,
 'eq1': b/6743 - e - 0.095699243660092,
 'eq2': c/6743 - e - 0.095699243660092,
 'eq3': d/6743 - e - 0.0678184784220673,
 'eq4': -e + f/6743 - 0.0384546937564882,
 'eq5': -e + g/6743 - 0.0308912946759603,
 'eq6': a + b + c + d + f + g - 2060}

In [18]:
# Calculate minimal error for investments with a deficit greater than zero
# The equation for e is derived from the system of equations presented above
cv = df['current_value'].to_list()
t_pct = df['target%'].to_list()
deficit = df['deficit'].to_list()
select_ranks = [i for i in range(len(df)) if deficit[i]>0]
select_cv = sum([cv[i] for i in select_ranks])
select_t_pct = sum([t_pct[i] for i in select_ranks])
select_ranks_cnt = len(select_ranks)
e = 1/select_ranks_cnt * ((select_cv + cash)/(sum(cv) + cash) - select_t_pct)

# Calculate cash allotments in a dictionary where keys are ranks and values are allotments
# The equation for each allotment is derived from the system of equations presented above
allotments = {i:((e + t_pct[i])*(sum(cv)+cash)) - cv[i] if i in select_ranks else 0 for i in range(len(df))}

# Create column containing allotments calculated via Method 2
df['allocate_for_minimal_errors'] = df.apply(lambda r: allocations[r['rank']],axis=1)

# Calculate errors based on Method 2 allotments
df['error3'] = ((df['current_value'] + df['allocate_for_minimal_errors']) / (df['current_value'].sum()+cash)) - df['target%']

df

,target%,current_value,current%,target_value,deficit,error,rank,allocate_by_rank,error2,allocate_for_minimal_errors,error3
7,0.1,12,0.002562,674.3,662.3,-0.097438,0,662.3,0.000000,526.0,-0.020214
1,0.1,29,0.006193,674.3,645.3,-0.093807,1,645.3,0.000000,509.0,-0.020214
2,0.1,29,0.006193,674.3,645.3,-0.093807,2,645.3,0.000000,509.0,-0.020214
6,0.1,217,0.046338,674.3,457.3,-0.053662,3,107.1,-0.051935,321.0,-0.020214
3,0.1,415,0.088618,674.3,259.3,-0.011382,4,0.0,-0.038455,123.0,-0.020214
5,0.1,466,0.099509,674.3,208.3,-0.000491,5,0.0,-0.030891,72.0,-0.020214
0,0.1,796,0.169977,674.3,-121.7,0.069977,6,0.0,0.018048,0.0,0.018048
8,0.1,848,0.181081,674.3,-173.7,0.081081,7,0.0,0.025760,0.0,0.025760
9,0.1,910,0.194320,674.3,-235.7,0.094320,8,0.0,0.034955,0.0,0.034955
4,0.1,961,0.205210,674.3,-286.7,0.105210,9,0.0,0.042518,0.0,0.042518


As shown in the output above, Method 1 and Method 2 result in different allotment sets.  By comparing the error columns, we can see that Method 2 optimizes errors across investments by minimizing them as much as possible (across investments with a deficit greater than zero).  

While the portfolio size and allotment amounts are relatively small in this example, the difference between optimal and suboptimal allocation sets may have significant consequences for larger portfolios with larger allocation amounts.  In high stakes situations involving hedge funds, company-owned assests, government or corporate budgets, or philanthropic initiatives, 
each allocation method could differentially impact overall portfolio performance or the recipients of the funding determined by the allocation method.

# Demonstration of portfolio_allocator()

In [4]:
%run "C:\Users\mcove\Downloads\portfolio_allocator().py"
portfolio_allocator(cash,current_values,target_pcts)

Method 1 is suboptimal.


,target%,current_value,current%,target_value,deficit,error,rank,allocate_by_rank,error2,allocate_for_minimal_errors,error3
0,0.1,12,0.002562,674.3,662.3,-0.097438,0,662.3,0.000000,526.0,-0.020214
1,0.1,29,0.006193,674.3,645.3,-0.093807,1,645.3,0.000000,509.0,-0.020214
2,0.1,29,0.006193,674.3,645.3,-0.093807,2,645.3,0.000000,509.0,-0.020214
3,0.1,217,0.046338,674.3,457.3,-0.053662,3,107.1,-0.051935,321.0,-0.020214
4,0.1,415,0.088618,674.3,259.3,-0.011382,4,0.0,-0.038455,123.0,-0.020214
5,0.1,466,0.099509,674.3,208.3,-0.000491,5,0.0,-0.030891,72.0,-0.020214
6,0.1,796,0.169977,674.3,-121.7,0.069977,6,0.0,0.018048,0.0,0.018048
7,0.1,848,0.181081,674.3,-173.7,0.081081,7,0.0,0.025760,0.0,0.025760
8,0.1,910,0.194320,674.3,-235.7,0.094320,8,0.0,0.034955,0.0,0.034955
9,0.1,961,0.205210,674.3,-286.7,0.105210,9,0.0,0.042518,0.0,0.042518
